In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, desc, col
from pyspark.sql.types import IntegerType
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

In [2]:
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("Twitch_Networks.com") \
    .getOrCreate()

22/07/18 19:00:39 WARN Utils: Your hostname, ELISABETH-PERSO resolves to a loopback address: 127.0.1.1; using 10.33.4.168 instead (on interface wlp1s0)
22/07/18 19:00:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/18 19:00:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Import des CSVs

In [3]:
df_top = spark.read.csv("hdfs://127.0.0.1:9000/esgi/data/projets/final/twitchdata-update.csv", 
                           header='true')
df_edges = spark.read.csv("hdfs://127.0.0.1:9000/esgi/data/projets/final/EDGELIST.csv", 
                           header='true')

#### cast string en int

In [4]:
df_top = df_top.withColumn("Followers",col("Followers").cast("integer"))
df_top = df_top.withColumn("Peak viewers",col("Peak viewers").cast("integer"))
df_top = df_top.withColumn("Average viewers",col("Average viewers").cast("integer"))
df_top = df_top.withColumn("Watch time(Minutes)",col("Watch time(Minutes)").cast("integer"))
df_top = df_top.withColumn("Stream time(minutes)",col("Stream time(minutes)").cast("integer"))
df_top = df_top.withColumn("Followers gained",col("Followers gained").cast("integer"))
df_top = df_top.withColumn("Views gained",col("Views gained").cast("integer"))
df_edges = df_edges.withColumn("Weight",col("Weight").cast("integer"))

### Edges streamers abonnés

In [5]:
df_edges.sort("Source").limit(5).toPandas()

,Source,Target,Weight
0,01bd02,PUBGKorea,922
1,01bd02,ch1ckenkun,2176
2,01bd02,pikra10,3376
3,01bd02,woowakgood,3290
4,01bd02,lol_woolf,1080


### Streamers les plus suivis

In [11]:
df_edges.groupBy("Target").count().sort(desc("count")).limit(20).toPandas()

,Target,count
0,Indiefoxx,1541
1,loltyler1,1342
2,ESL_CSGO,1121
3,Rainbow6,1074
4,Asmongold,966
5,RocketLeague,932
6,iiTzTimmy,902
7,TenZ,878
8,aceu,870
9,ludwig,866


#### ceux qui offrent le plus de subscribers

In [12]:
df_edges.sort(desc("Weight")).limit(20).toPandas()

,Source,Target,Weight
0,RanbooLive,tommyinnit,437725
1,RanbooLive,Tubbo,423393
2,ibai,auronplay,423011
3,Tubbo,tommyinnit,359039
4,RanbooLive,TubboLIVE,333957
5,xQcOW,summit1g,333188
6,Quackity,RanbooLive,331202
7,ibai,Rubius,330622
8,RanbooLive,WilburSoot,315742
9,ibai,TheGrefg,309730


### Moyenne des followers des top streamers

# pas utiliser collect -> machine locale

In [8]:
df_top.select(mean("Followers")).toPandas()

,avg(Followers)
0,570054.068


quartile
mediane
engagement abonne/watch time
relation entre streamers
cluster de chaine

### Streamers par ordre décroissant de vues simultanées

In [9]:
df_top.sort(desc("Peak viewers")).toPandas()

,Channel,Watch time(Minutes),Stream time(minutes),Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language
0,Riot Games (riotgames),NaN,80820,639375,20960,4487489,497678,56855694,True,False,English
1,TheGrefg,1.757407e+09,54855,538444,28887,3795667,3593081,47094362,True,False,Spanish
2,dota2ti,1.017578e+09,6315,483530,147643,663297,121422,16228039,True,False,English
3,shroud,8.885052e+08,30240,471281,29612,7744066,833587,30621257,True,False,English
4,dota2ti_ru,8.125381e+08,6195,457060,126232,541644,108438,12068376,True,False,Russian
...,...,...,...,...,...,...,...,...,...,...,...
995,voicetv,1.625107e+08,514845,1874,314,3660,1328,4001475,True,False,Thai
996,SaltyTeemo,2.476133e+08,517980,1803,476,173196,36103,2117741,False,False,English
997,MarketTradersTV,1.489001e+08,499215,1357,297,30413,12081,981536,True,False,English
998,runitup247,2.057460e+08,486000,1182,422,91608,30397,2424188,True,False,English


### Streamers par ordre décroissant de followers

In [10]:
df_top.sort(desc("Followers")).toPandas()

,Channel,Watch time(Minutes),Stream time(minutes),Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language
0,Tfue,NaN,123660,285644,29602,8938903,2068424,78998587,True,False,English
1,shroud,8.885052e+08,30240,471281,29612,7744066,833587,30621257,True,False,English
2,Myth,1.479215e+09,134760,122552,9396,6726893,1421811,37384058,True,False,English
3,Rubius,NaN,58275,240096,42948,5751354,3820532,58599449,True,False,Spanish
4,pokimane,9.643341e+08,56505,112160,16026,5367605,2085831,45579002,True,False,English
...,...,...,...,...,...,...,...,...,...,...,...
995,칸데르니아 (jmjdoc),1.314082e+08,170865,2395,756,30877,9502,4337476,True,False,Korean
996,MarketTradersTV,1.489001e+08,499215,1357,297,30413,12081,981536,True,False,English
997,NiteNightKid,1.784190e+08,149040,29228,1127,28818,11998,1450208,True,True,English
998,Sinner666,1.248122e+08,24765,27996,4530,7102,7101,21322548,False,False,Russian


standardize the data to have a mean of zero and a standard deviation of one.
#### kmeans

In [ ]:
cost = np.zeros(20)
for k in range(2,20):
    kmeans = KMeans()\
            .setK(k)\
            .setSeed(1) \
            .setFeaturesCol("indexedFeatures")\
            .setPredictionCol("cluster")

    model = kmeans.fit(data)
    cost[k] = model.computeCost(data) 